## Profitable App Profiles for the App Store and Google Play Markets

For this project, i pretend working as a data analyst for a company that builds Android and iOS mobile apps. We make our apps available on Google Play and in the App Store.

We only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that the number of users of our apps determines our revenue for any given app — the more users who see and engage with the ads, the better. 

My goal for this project is to analyze data to help our developers understand what type of apps are likely to attract more users.

1. Imports:

In [1]:
from csv import reader

In [2]:
android_file = open("googleplaystore.csv")
android_data = reader(android_file)
android_data_listed = list(android_data)
android_file.close()

In [3]:
ios_file = open("AppleStore.csv")
ios_data = reader(ios_file)
ios_data_listed = list(ios_data)
ios_file.close()

2. Printing basic info about data:

In [4]:
def explore_data(dataset, start, end, rows_and_columns = False):
    dataset_sliced = dataset[start:end]
    for row in dataset_sliced:
        print(row)
        print("\n")
        
    if rows_and_columns:
        print("Number of rows:", len(dataset))
        print("Number of columns:", len(dataset[0]))

In [5]:
print(explore_data(android_data_listed, 0, 6, rows_and_columns=True))

In [6]:
print(explore_data(ios_data_listed, 0, 6, rows_and_columns=True))

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 7198
Number of columns: 16
None


3. Data cleaning based on info about dataset:

In [7]:
print(android_data_listed[10473])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


In [8]:
del android_data_listed[10473]

4. Removing duplicates:

In [9]:
unique_names = []
duplicated_names = []

for row in android_data_listed:
    app_name = row[0]
    if app_name in unique_names:
        duplicated_names.append(app_name)
    else:
        unique_names.append(app_name)

In [10]:
print("Number of duplicated apps in Android dataset: ", len(duplicated_names))
print("Examples of duplicated apps in Android dataset: ", duplicated_names[:10])

Number of duplicated apps in Android dataset:  1181
Examples of duplicated apps in Android dataset:  ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


In [11]:
reviews_max = {}

for row in android_data_listed[1:]:
    app_name = row[0]
    n_reviews = float(row[3])
    if app_name in reviews_max and reviews_max[app_name] < n_reviews:
        reviews_max[app_name] = n_reviews
    elif app_name not in reviews_max:
        reviews_max[app_name] = n_reviews

In [12]:
android_data_listed_clean = []
already_added = []

for row in android_data_listed[1:]:
    app_name = row[0]
    n_reviews = float(row[3])
    if n_reviews == reviews_max[app_name] and app_name not in already_added:
        android_data_listed_clean.append(row)
        already_added.append(app_name)
        
print(len(android_data_listed_clean))
print(len(already_added))

9659
9659


5. Removing non-english Apps:

In [13]:
def ascii_checker(word):
    counter = 0
    for letter in word:
        if ord(letter) > 127:
            counter += 1
    if counter > 3:
        return False
    else:
        return True

In [14]:
android_data_listed_clean_english = []
ios_data_listed_english = []

def english_apps_checker(dataset, index, english_list):
    for row in dataset:
        app_name = row[index]
        if  ascii_checker(app_name):
            english_list.append(row)
            
english_apps_checker(android_data_listed_clean, 0, android_data_listed_clean_english)
english_apps_checker(ios_data_listed, 1, ios_data_listed_english)

In [15]:
print(len(android_data_listed_clean_english))
print(len(ios_data_listed_english))

9614
6184


6. Isolating the free Apps:

In [16]:
android_apps_final = []
ios_apps_final = []

def free_apps_separator(dataset, index, status, free_apps_list):
    for row in dataset:
        price = row[index]
        if price == status:
            free_apps_list.append(row)

free_apps_separator(android_data_listed_clean_english, 7, "0", android_apps_final)
free_apps_separator(ios_data_listed_english, 4, "0.0", ios_apps_final)

In [17]:
print(len(android_apps_final))
print(len(ios_apps_final))

8864
3222


7. Counting most common Apps per Genre:

In [18]:
def freq_table(dataset, index):
    freq_dict = {}
    total = 0
    
    for row in dataset:
        total += 1
        key = row[index]
        if key in freq_dict:
            freq_dict[key] += 1
        else:
            freq_dict[key] = 1
    
    perc_dict = {}
    for key in freq_dict:
        percentage = (freq_dict[key] / total) * 100
        perc_dict[key] = percentage
    
    return perc_dict

In [19]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
        
    return table_display

In [20]:
android_apps_per_genre = display_table(android_apps_final, 9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

In [21]:
android_apps_per_category = display_table(android_apps_final, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [22]:
ios_apps_per_genre = display_table(ios_apps_final, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


- Regarding iOS: over 58% of the Apps are games with education being on 4th place with 3.66% only.
- Regarding Android: Tools are the only category / genre spotted in top 3 for both lists.
- Regarding Android: only Family category can be treated as a leader.
- Overall: it's all not about how popular these apps are but how many of them are in a certain category.

8. Most Popular Apps by Genre on the App Store:

In [23]:
for genre in ios_apps_per_genre:
    total = 0
    len_genre = 0
    for row in ios_apps_final:
        genre_app = row[11]
        if genre_app == genre[1]:
            n_ratings = float(row[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = round(total / len_genre, 2)
    print(genre[1], ":", avg_n_ratings)

Social Networking : 71548.35
Photo & Video : 28441.54
Games : 22788.67
Music : 57326.53
Reference : 74942.11
Health & Fitness : 23298.02
Weather : 52279.89
Utilities : 18684.46
Travel : 28243.8
Shopping : 26919.69
News : 21248.02
Navigation : 86090.33
Lifestyle : 16485.76
Entertainment : 14029.83
Food & Drink : 33333.92
Sports : 23008.9
Book : 39758.5
Finance : 31467.94
Education : 7003.98
Productivity : 21028.41
Business : 7491.12
Catalogs : 4004.0
Medical : 612.0


9. Most Popular Apps by Genre on Google Play:

In [24]:
for category in android_apps_per_category:
    total = 0
    len_category = 0
    for row in android_apps_final:
        category_app = row[1]
        if category_app == category[1]:
            n_installs = row[5]
            n_installs = n_installs.replace(",", "")
            n_installs = n_installs.replace("+", "")
            total += float(n_installs)
            len_category += 1
    n_installs_avg = round(total / len_category, 2)
    print(category[1], ":", n_installs_avg)

ART_AND_DESIGN : 1986335.09
AUTO_AND_VEHICLES : 647317.82
BEAUTY : 513151.89
BOOKS_AND_REFERENCE : 8767811.89
BUSINESS : 1712290.15
COMICS : 817657.27
COMMUNICATION : 38456119.17
DATING : 854028.83
EDUCATION : 1833495.15
ENTERTAINMENT : 11640705.88
EVENTS : 253542.22
FINANCE : 1387692.48
FOOD_AND_DRINK : 1924897.74
HEALTH_AND_FITNESS : 4188821.99
HOUSE_AND_HOME : 1331540.56
LIBRARIES_AND_DEMO : 638503.73
LIFESTYLE : 1437816.27
GAME : 15588015.6
FAMILY : 3695641.82
MEDICAL : 120550.62
SOCIAL : 23253652.13
SHOPPING : 7036877.31
PHOTOGRAPHY : 17840110.4
SPORTS : 3638640.14
TRAVEL_AND_LOCAL : 13984077.71
TOOLS : 10801391.3
PERSONALIZATION : 5201482.61
PRODUCTIVITY : 16787331.34
PARENTING : 542603.62
WEATHER : 5074486.2
VIDEO_PLAYERS : 24727872.45
NEWS_AND_MAGAZINES : 9549178.47
MAPS_AND_NAVIGATION : 4056941.77
